# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. 

In [2]:
# Your code here:
data = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
# Your code here:
data.columns = map(lambda x: x.lower().replace("-", "_").replace(" ", "_"), data.columns)
data.head()

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate
0,"ADRIANO, RACQUEL ANNE",PARAMEDIC,FIRE,F,Salary,NaN,72510.0,NaN
1,"AFFANEH, MAHIR A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
2,"AFOENYI, PHYLLIS",TITLE V PROGRAM TRAINEE I,FAMILY & SUPPORT,P,Hourly,20.0,NaN,13.0
3,"AGAR, BULENT B",DEPUTY COMMISSIONER,WATER MGMNT,F,Salary,NaN,132972.0,NaN
4,"AGEE, LAWRENCE",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
data.shape

(33702, 8)

In [6]:
# Your code here:
data.isna().sum()

name                     0
job_titles               0
department               0
full_or_part_time        0
salary_or_hourly         0
typical_hours        25528
annual_salary         8174
hourly_rate          25528
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [7]:
# Your code here:
data['salary_or_hourly'].value_counts(dropna=False)

Salary    25528
Hourly     8174
Name: salary_or_hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [8]:
# Your code here:
data.pivot_table(index=['department'], values=['name'], aggfunc=({'name':pd.Series.nunique})).reset_index()

,department,name
0,ADMIN HEARNG,38
1,ANIMAL CONTRL,75
2,AVIATION,1855
3,BOARD OF ELECTION,111
4,BOARD OF ETHICS,8
5,BUDGET & MGMT,44
6,BUILDINGS,258
7,BUSINESS AFFAIRS,167
8,CITY CLERK,88
9,CITY COUNCIL,362


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [9]:
hourly_workers = data.dropna(subset=['hourly_rate']).reset_index()
hourly_workers_crop = hourly_workers['hourly_rate']

In [10]:
from scipy.stats import ttest_1samp

stat, pval = ttest_1samp(hourly_workers_crop, 30)

In [11]:
# Two-sided test:
# Null hypothesis or H0: hourly wage = 30
# Alternative hyp or H1: hourly wage <> 30

print('stat is  ', stat)
print('pvalue for the two-tailed test is ', pval)
print('pvalue for the one-tailed test is ', pval/2)

stat is   31.211500329216044
pvalue for the two-tailed test is  3.825475847673794e-202
pvalue for the one-tailed test is  1.912737923836897e-202


In [13]:
# # nothing to do.
# # percentage of difference
np.mean(hourly_workers_crop)

34.58644971862046

#### Conclusion : 
with two sided test, p value almost zero. So we reject H0, which is hourly wage is 30.
We assume hourly pay is significantly different than 30

----------------------------------------------

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [14]:
# Your code here:
police_salary = data[(data['department']=='POLICE')&(data['annual_salary'].isna()==False)]
police_salary_crop = police_salary['annual_salary']

In [18]:
stat, pval = ttest_1samp(police_salary_crop, 86000)

# One-sided test:
# Null hypothesis or H0: police salary >= 86000
# Alternative hyp or H1: police salary < 86000

print('stat is  ', stat)
print('pvalue for the two-tailed test is ', pval)
print('pvalue for the one-tailed test is ', pval/2)

stat is   18.51748561981861
pvalue for the two-tailed test is  1.2239133904242963e-75
pvalue for the one-tailed test is  6.119566952121482e-76


#### Conclusion:
As p value is almost zero, salary is not equal to 86K. When we look at the stat, it is positive and we can say salary is higher than 86K.

---------------------------------------------------------------------

Using the `crosstab` function, find the department that has the most hourly workers. 

In [19]:
# Your code here:
data_crosstab = pd.crosstab(data['department'], data['hourly_rate'], margins=True)
most_hourly_workers = pd.DataFrame(data_crosstab.sort_values(by='All',ascending=False)).reset_index()
most_hourly_workers.loc[[1],['department']]

hourly_rate,department
1,STREETS & SAN


#### Conclusion: 
STREET&SAN has the most hourly workers

-------------------------------------------------------------

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [20]:
street_and_san = data[(data['department']=='STREETS & SAN')&(data['hourly_rate'].isna()==False)]
street_and_san_crop = street_and_san['hourly_rate']

In [21]:
# One-sided test:
# Null hypothesis or H0: hourly_wage >= 35$
# Alternative hyp or H1: hourly_wage < 35$

stat, pval = ttest_1samp(street_and_san_crop, 35)
print('stat is  ', stat)
print('pvalue for the two-tailed test is ', pval)
print('pvalue for the one-tailed test is ', pval/2)

stat is   4.000138803968834
pvalue for the two-tailed test is  6.580474443773579e-05
pvalue for the one-tailed test is  3.2902372218867895e-05


#### Conclusion : 
As per our p value, we reject that salary is equal 35 per hour.
Per stat, it is greater than 35. We accept that hourly wage is greater than 35 per hour.

-----------------------------------------------------------------

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [22]:
hourly_rate = data[data['hourly_rate'].isna()==False]
hourly_rate_crop = pd.DataFrame(hourly_rate['hourly_rate'])

In [23]:
import scipy.stats
confidence_level = 0.95 
degrees_freedom = len(hourly_rate_crop) - 1  
sample_mean = np.mean(hourly_rate_crop)
sample_standard_error = scipy.stats.sem(hourly_rate_crop)

confidence_interval = scipy.stats.t.interval(confidence_level, 
                                             degrees_freedom, 
                                             sample_mean, 
                                             sample_standard_error)
print( 'confidence interval is ', confidence_interval, '.' )

confidence interval is  (array([34.29839539]), array([34.87450405])) .


C:\Users\aslis\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [24]:
# police_salary = data[(data['department']=='POLICE')&(data['annual_salary'].isna()==False)]
# police_salary_crop = police_salary['annual_salary']
import scipy.stats

confidence_level = 0.95 
degrees_freedom = len(police_salary_crop) - 1  
sample_mean = np.mean(police_salary_crop)
sample_standard_error = scipy.stats.sem(police_salary_crop)

confidence_interval = scipy.stats.t.interval(confidence_level, 
                                             degrees_freedom, 
                                             sample_mean, 
                                             sample_standard_error)
print( 'confidence interval is ', confidence_interval, '.' )

confidence interval is  (88534.1182885883, 89134.11955863392) .


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

We can assume it is not significally different than %25

In [25]:
len(hourly_workers_crop)

8174

In [13]:
# Your code here:
# import library
from statsmodels.stats.proportion import proportions_ztest
  
# perform one proportion z-test
proportions_ztest(count= 8174, nobs=33702, value=0.25, alternative='larger')[1]


0.9993038567647838